Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement json (from versions: none)
ERROR: No matching distribution found for json


Here is a list of all the JSON elements from the Works API we are getting (spelling not perfect):
id
title
display name
pub year
pub date
type
type_crossref
authorships.
	author position:
	author.
		id
	institutions.
		id
		display name
		country code
		type
	countries (list)
countries_distinct_count
institutions_distinct_count
cited by count
citation normalized percentile
	value
primary topic
	id
	display name
	score
	subfield.
		id
		display name
	field
		id
		display name
	domain
		id
		display name
topics (list)
		id
	display name
	score
	subfield.
		id
		display name
	field.
		id
		display name
	domain.
		id
		display name
keywords (list)
	id
	display name
	score
concepts (list)
	id
	display name
	level
	score

referenced works (list)
related works (list)



In [22]:
import requests
import pandas as pd
import json


"""
Fetch works from the OpenAlex API and return a DataFrame.

Args:
    per_page (int): Number of results per page (max 200).
    search (str): Optional search query to filter works.

Returns:
    pd.DataFrame: DataFrame containing works data.
"""
base_url = "https://api.openalex.org/works/W2741809807"


works = requests.get(base_url)
works.raise_for_status()

print(json.dumps(works.json(), indent=4))



{
    "id": "https://openalex.org/W2741809807",
    "doi": "https://doi.org/10.7717/peerj.4375",
    "title": "The state of OA: a large-scale analysis of the prevalence and impact of Open Access articles",
    "display_name": "The state of OA: a large-scale analysis of the prevalence and impact of Open Access articles",
    "publication_year": 2018,
    "publication_date": "2018-02-13",
    "ids": {
        "openalex": "https://openalex.org/W2741809807",
        "doi": "https://doi.org/10.7717/peerj.4375",
        "mag": "2741809807",
        "pmid": "https://pubmed.ncbi.nlm.nih.gov/29456894",
        "pmcid": "https://www.ncbi.nlm.nih.gov/pmc/articles/5815332"
    },
    "language": "en",
    "primary_location": {
        "is_oa": true,
        "landing_page_url": "https://doi.org/10.7717/peerj.4375",
        "pdf_url": "https://peerj.com/articles/4375.pdf",
        "source": {
            "id": "https://openalex.org/S1983995261",
            "display_name": "PeerJ",
            "issn

In [8]:
pip install gzip

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement gzip (from versions: none)

[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: C:\Users\jeffr\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip
ERROR: No matching distribution found for gzip


In [58]:
import json
import zipfile
import gzip
import pandas as pd
from tabulate import tabulate
import os
import datetime

start_date = datetime.datetime.strptime("2023-05-17", "%Y-%m-%d")
end_date = datetime.datetime.strptime("2023-05-30", "%Y-%m-%d")
cur_date = start_date
while cur_date <= end_date:
    folder = "C:\\Users\\jeffr\\Documents\\openalex-snapshot\\updated_date=" + cur_date.strftime("%Y-%m-%d")
    cur_date += datetime.timedelta(days=1)
    try: 
        # print(os.listdir(folder))

        for filename in os.listdir(folder):
            if filename.endswith(".gz"):
                print(os.path.join(folder, filename))

                works = pd.DataFrame()
                authors = pd.DataFrame()
                topics = pd.DataFrame()
                keywords = pd.DataFrame()
                concepts = pd.DataFrame()
                referenced_works = pd.DataFrame()

                with gzip.open(os.path.join(folder, filename)) as data_file:
                    for line in data_file:
                        id = json.loads(line)
                        
                        work = pd.DataFrame()
                        work["id"] = [id.get("id")]
                        work["pub_year"] = [id.get("publication_year")]
                        work["pub_date"] = [id.get("publication_date")]
                        # source = id.get("primary_location,", {}).get("source", {}).get("type")
                        work["is_published"] = [id.get("primary_location,", {}).get("is_published")]
                        work["type"] = [id.get("type")]
                        work["type_crossref"] = [id.get("type_crossref")]


                        
                        for author in id.get("authorships", []):
                            row = pd.DataFrame()
                            row["work_id"] = [id.get("id", "")]
                            row["a_id"] = [author["author"].get("id", "")]
                            row["position"] = [author.get("author_position", "")]
                            row["countries"] = [author.get("countries", [])]
                            authors = pd.concat([authors, row], ignore_index=True)

                        work["cited_by_count"] = [id.get("cited_by_count")]
                        work["citation_normalized_percentile"] = [id.get("citation_normalized_percentile", {}).get("value", "")]

                        for topic in id.get("topics", []):
                            row = pd.DataFrame()
                            row["work_id"] = [id.get("id", "")]
                            row["topic_id"]  = [topic.get("id", "")]
                            row["topic_display_name"]  = [topic.get("display_name", "")]
                            row["topic_score"]  = [topic.get("score", "")]
                            row["topic_sub_field_display_name"]  = [topic.get("subfield", {}).get("display_name", "")]
                            row["topic_field_display_name"]  = [topic.get("field", {}).get("display_name", "")]
                            row["topic_domain_display_name"]  = [topic.get("domain", {}).get("display_name", "")]
                            topics = pd.concat([topics, row], ignore_index=True)

                        for keyword in id.get("keywords", []):  
                            row = pd.DataFrame()
                            row["work_id"] = [id.get("id", "")]
                            row["keyword_id"] = [keyword.get("id", "")]
                            row["keyword_name"] = [keyword.get("display_name", "")]
                            row["keyword_score"] = [keyword.get("score", "")]
                            keywords = pd.concat([keywords, row], ignore_index=True)

                        for concept in id.get("concepts", []):
                            row = pd.DataFrame()
                            row["work_id"] = [id.get("id", "")]
                            row["concept_id"] = [concept.get("id", "")]
                            row["concept_display_name"] = [concept.get("display_name", "")]
                            row["concept_level"] = [concept.get("level", "")]
                            row["concept_score"] = [concept.get("score", "")]
                            concepts = pd.concat([concepts, row], ignore_index=True)

                        for r_work in id.get("referenced_works", []):
                            row = pd.DataFrame()
                            row["work_id"] = [id.get("id", "")]
                            row["referenced_work_id"] = [r_work]
                            referenced_works = pd.concat([referenced_works, row], ignore_index=True)
                        


                        works = pd.concat([works, work], ignore_index=True)
                        # print(work.head())

                # extract info into csv
                works.to_csv(folder + "\\works.csv", index=False)
                authors.to_csv(folder + "\\authors.csv", index=False)
                topics.to_csv(folder + "\\topics.csv", index=False)
                keywords.to_csv(folder + "\\keywords.csv", index=False)
                concepts.to_csv(folder + "\\concepts.csv", index=False)
                referenced_works.to_csv(folder + "\\referenced_works.csv", index=False)
                # print(tabulate(authors, headers='keys', tablefmt='psql'))

                # delete raw file
                # os.remove(os.path.join(folder, filename))


    except FileNotFoundError:
        print(f"Folder not found: {folder}")




C:\Users\jeffr\Documents\openalex-snapshot\updated_date=2023-05-17\part_000.gz
Folder not found: C:\Users\jeffr\Documents\openalex-snapshot\updated_date=2023-05-18
Folder not found: C:\Users\jeffr\Documents\openalex-snapshot\updated_date=2023-05-19
Folder not found: C:\Users\jeffr\Documents\openalex-snapshot\updated_date=2023-05-20
Folder not found: C:\Users\jeffr\Documents\openalex-snapshot\updated_date=2023-05-21
Folder not found: C:\Users\jeffr\Documents\openalex-snapshot\updated_date=2023-05-22
Folder not found: C:\Users\jeffr\Documents\openalex-snapshot\updated_date=2023-05-23
Folder not found: C:\Users\jeffr\Documents\openalex-snapshot\updated_date=2023-05-24
Folder not found: C:\Users\jeffr\Documents\openalex-snapshot\updated_date=2023-05-25
Folder not found: C:\Users\jeffr\Documents\openalex-snapshot\updated_date=2023-05-26
Folder not found: C:\Users\jeffr\Documents\openalex-snapshot\updated_date=2023-05-27
Folder not found: C:\Users\jeffr\Documents\openalex-snapshot\updated_da

In [ ]:
import datetime
import os


start_date = datetime.datetime.strptime("2023-05-17", "%Y-%m-%d")
end_date = datetime.datetime.strptime("2025-09-19", "%Y-%m-%d")
cur_date = start_date
while cur_date <= end_date:
    folder = "C:\\Users\\jeffr\\Documents\\openalex-snapshot\\updated_date=" + cur_date.strftime("%Y-%m-%d")
    cur_date += datetime.timedelta(days=1)
    try: 
        # print(os.listdir(folder))

        for filename in os.listdir(folder):
            if filename.endswith(".gz"):
                print(os.path.join(folder, filename))
    except FileNotFoundError:
        print(f"Folder not found: {folder}")


_IncompleteInputError: incomplete input (622446149.py, line 18)

In [ ]:
works = pd.DataFrame()
authors = pd.DataFrame()
topics = pd.DataFrame()
keywords = pd.DataFrame()
concepts = pd.DataFrame()
referenced_works = pd.DataFrame()
folder = "C:\\Users\\jeffr\\Documents\\test_csvs"
with gzip.open("C:\\Users\\jeffr\\Documents\\openalex-snapshot\\updated_date=2025-09-16\\part_000.gz") as data_file:
    for line in data_file:
        id = json.loads(line)
        
        work = pd.DataFrame()
        work["id"] = [id.get("id", "")]
        work["pub_year"] = [id.get("publication_year", "")]
        work["pub_date"] = [id.get("publication_date", "")]
        # source = id.get("primary_location,", {}).get("source", {}).get("type")
        work["is_published"] = [id.get("primary_location,", {}).get("is_published")]
        work["type"] = [id.get("type")]
        work["type_crossref"] = [id.get("type_crossref")]


        
        for author in id.get("authorships", []):
            row = pd.DataFrame()
            row["work_id"] = [id.get("id", "")]
            row["a_id"] = [author["author"].get("id", "")]
            row["position"] = [author.get("author_position", "")]
            row["countries"] = [author.get("countries", [])]
            authors = pd.concat([authors, row], ignore_index=True)

        work["cited_by_count"] = [id.get("cited_by_count")]
        # work["citation_normalized_percentile"] = [id.get("citation_normalized_percentile", {}).get("value", "")]

        for topic in id.get("topics", []):
            row = pd.DataFrame()
            row["work_id"] = [id.get("id", "")]
            row["topic_id"]  = [topic.get("id", "")]
            row["topic_display_name"]  = [topic.get("display_name", "")]
            row["topic_score"]  = [topic.get("score", "")]
            row["topic_sub_field_display_name"]  = [topic.get("subfield", {}).get("display_name", "")]
            row["topic_field_display_name"]  = [topic.get("field", {}).get("display_name", "")]
            row["topic_domain_display_name"]  = [topic.get("domain", {}).get("display_name", "")]
            topics = pd.concat([topics, row], ignore_index=True)

        for keyword in id.get("keywords", []):  
            row = pd.DataFrame()
            row["work_id"] = [id.get("id", "")]
            row["keyword_id"] = [keyword.get("id", "")]
            row["keyword_name"] = [keyword.get("display_name", "")]
            row["keyword_score"] = [keyword.get("score", "")]
            keywords = pd.concat([keywords, row], ignore_index=True)

        for concept in id.get("concepts", []):
            row = pd.DataFrame()
            row["work_id"] = [id.get("id", "")]
            row["concept_id"] = [concept.get("id", "")]
            row["concept_display_name"] = [concept.get("display_name", "")]
            row["concept_level"] = [concept.get("level", "")]
            row["concept_score"] = [concept.get("score", "")]
            concepts = pd.concat([concepts, row], ignore_index=True)

        for r_work in id.get("referenced_works", []):
            row = pd.DataFrame()
            row["work_id"] = [id.get("id", "")]
            row["referenced_work_id"] = [r_work]
            referenced_works = pd.concat([referenced_works, row], ignore_index=True)
        


        works = pd.concat([works, work], ignore_index=True)
        # print(work.head())

# extract info into csv
works.to_csv(folder + "\\works.csv", index=False)
authors.to_csv(folder + "\\authors.csv", index=False)
topics.to_csv(folder + "\\topics.csv", index=False)
keywords.to_csv(folder + "\\keywords.csv", index=False)
concepts.to_csv(folder + "\\concepts.csv", index=False)
referenced_works.to_csv(folder + "\\referenced_works.csv", index=False)
# print(tabulate(authors, headers='keys', tablefmt='psql'))

# delete raw file
# os.remove(os.path.join(folder, filename))

AttributeError: 'NoneType' object has no attribute 'get'